# Introduction

This tutorial will walk you through a preliminary similarity searching analysis making use of scripts in the AMOEBAE toolkit. While AMOEBAE was not originally written to be used via the command line, Jupyter notebooks provide an easy means of guiding new users through an example analysis with limited need for manual input. The end result of running this code successfully is a spreadsheet summarizing results of similarity searches, as well as a plot to visualize the results.

As a simple example, we will consider the the distribution of orthologues of subunits of the Adaptor Protein (AP) 2 vesicle adaptor complex, and several other membrane-trafficking proteins, in five model eukaryotes: the plant *Arabidopsis thaliana*, the yeast *Saccharomyces cerevisiae*, the fungus *Allomyces macrogynus*, the amoeba *Dictyostelium discoideum*, and the pathogenic protist *Trypanosoma brucei*. AP-2 subunits are homologous to subunits of other AP complexes (Robinson, 2004; Hirst et al., 2011), and published work has traced their evolution among plants (Larson et al., 2019), Fungi (Barlow et al., 2014), and trypanosomatid parasites (Manna et al., 2013). Thus, the protein subunits of the AP-2 complex provide a useful test of similarity searching methods to distinguish between orthologues and paralogues, which can be compared to the results of previous studies. In addition, the membrane trafficking proteins Sec12 (a component of the COPII vesicle coat complex), SNAP33 (a Qbc-SNARE), and Rab2 (a small GTPase) are included to further explore the potential sources of error involved in identification of orthologous proteins.

## Objectives of this tutorial


-  Perform similarity searches using the BLASTP, TBLASN, HMMer algorithms simultaneously using AMOBEAE scripts.

-  Apply a reciprocal-best-hit search strategy using AMOEBAE code.

- Practice interpreting similarity search results obtained using AMOEBAE.


## Requirements

- Before running this code, you will need to have set up AMOEBAE according to the instructions in the main documentation file here (which you likely have already done): [AMOEBAE_documentation.pdf](
https://github.com/laelbarlow/amoebae/blob/master/documentation/AMOEBAE_documentation.pdf).

- MacOS or Linux operating system (or possibly a work-around on windows, although this has not been tested).

- Approximately 3GB of storage space.

- An internet connection.

- At least an hour of your time (the code in this notebook will take approximately 60 minutes to run).

- Running the code in this notebook is more computationally intensive than webbrowsing for example, so if you are running this on a laptop computer, then make sure it is connected to an electrical outlet.

## Testing
If you wish to simply run all the code in this notebook for testing purposes:

- First, modify the cell in the section labeled "Enter your email to access the NCBI protein database via NCBI Entrez" below such that the value of Entrez.email is hard-coded as your email address.

- Then select "Cell" > "Run All" from the Jupyter menu above.

# Preliminary steps

## Find the amoebae script

The directory containing the amoebae executable script must be present in your $PATH.

In [ ]:
%%bash
printf "\nThis is the directory that this notebook is run in:\n"
pwd
echo
printf "\nThis is the path to the amoebae executable script that will be used:\n"
command -v amoebae
#echo
#printf "\nThese are all the paths in the \$PATH variable:\n"
#tr ':' '\n' <<< "$PATH"

## Check that dependencies are installed

You should have already pulled the amoebae git repository to your computer as described in the main documentation file.

In [ ]:
%%bash
# This command simply prints the versions of some dependencies which are now available for use by amoebae.
amoebae check_depend

In [ ]:
%%bash
# This command tests all the import statements in amoebae modules.
amoebae check_imports

## Import some basic python modules

In [ ]:
import os
import sys
import platform
import subprocess
from Bio import SeqIO
from Bio import Entrez
import glob
from Bio.Blast import NCBIXML
import pandas as pd
from IPython.display import display, HTML, Image

## Update PATH so that additional modules can be imported.

In [ ]:
# Add parent directory (the main amoebae repository directory) to the $PATH.
sys.path.append('..')
!echo $PATH

In [ ]:
import settings

## Record the specific version of AMOEBAE code used

In [ ]:
# Record git repository version information.
wd = !pwd
script_dir = wd[0] 
git_hash = str(subprocess.check_output(["git", "rev-parse", "HEAD"], cwd=script_dir).strip())
git_branch = str(subprocess.check_output(["git", "rev-parse", "--abbrev-ref", "HEAD"], cwd=script_dir).strip())  
print('\nGit repository (code) version: ' + git_hash + ' (branch name: ' + git_branch + ')\n')

## Make a subdirectory to store output.

In [ ]:
%%bash
mkdir amoebae_tutorial_2_output

In [ ]:
%cd amoebae_tutorial_2_output

## Initiate a data directory structure
To generate a directory structure and spreadsheets for storing formatted sequence files
and metadata for each sequence file, use the 'mkdatadir' command (this takes a
single argument which is the full path that you want your new directory to be
written to):

In [ ]:
%env DATADIR=AMOEBAE_Data

In [ ]:
%%bash
amoebae mkdatadir $DATADIR

This will prompt you to set the 'root\_amoebae\_data\_dir' variable in the
settings.py file to this new directory path so that AMOEBAE scripts can locate
your files.

This can be done as follows:

In [ ]:
# Check that the path indicated in the settings file is correct.
import settings
print(settings.root_amoebae_data_dir)
assert settings.root_amoebae_data_dir == "AMOEBAE_Data"

# Set up queries

## Enter your email to access the NCBI protein database via NCBI Entrez

In [ ]:
# Comment out this line and use the line at the bottom of this cell instead, if you want to run all cells at once.
#Entrez.email = input("Enter your email address here: ")  # Tell NCBI who you are.

# Use the line at the top of this cell instead.
#Entrez.email = "yourname@email.com"
Entrez.email = "lael@ualberta.ca"

## Download single-sequence queries

In [ ]:
%%time

# Define a dictionary with NCBI sequence accessions as keys and filenames to write
# the corresponding sequences to as values.
query_dict = {"NP_194077.1": "AP1beta_Athaliana_NP_194077.1_query.faa",
              "NP_851058.1": "AP2alpha_Athaliana_NP_851058.1_query.faa",
              "NP_974895.1": "AP2mu_Athaliana_NP_974895.1_query.faa",
              "NP_175219.1": "AP2sigma_Athaliana_NP_175219.1_query.faa",
              "NP_566961.1": "Sec12_Athaliana_NP_566961.1_query.faa",
              "NP_200929.1": "SNAP33_Athaliana_NP_200929.1_query.faa",
              "NP_193449.1": "Rab2_Athaliana_NP_193449.1_query.faa"
          }

# Make a new temporary directory to store sequence files.
temp_query_dir_name = 'temporary_query_dir'
if not os.path.isdir(temp_query_dir_name):
    os.mkdir(temp_query_dir_name)

# Loop over keys in the query_dict dictionary.
for accession in query_dict.keys():
    # Retrieve the corresponding filename from the dictionary.
    filename = query_dict[accession]
    filepath = os.path.join(temp_query_dir_name, filename)
    # Only download sequences that have not already been downloaded.
    if not os.path.isfile(filepath):
        # Download the sequence from NCBI via Entrez, using the Biopython module.
        net_handle = Entrez.efetch(db="protein", id=accession, rettype="fasta", retmode="text")
        out_handle = open(filepath, "w")
        out_handle.write(net_handle.read())
        out_handle.close()
        net_handle.close()
    # Check that the sequence was actually downloaded.
    assert os.path.isfile(filepath), """The sequence with the following accession could not be downloaded from NCBI: %s\n
    Try re-running this cell.""" % accession

## Prepare single-sequence queries for searching

Queries must be formatted and stored in a similar manner to genomic data files. The query files will include FASTA files containing one sequence and FASTA files containing multiple sequences.
Now we are going to generate the query files and add them to your AMOEBAE_Data/ Queries directory, in a similar way to how we added genomic data files to the AMOEBAE_Data/Genomes directory. Since you already downloaded all the peptide sequences for Arabidopsis thaliana, you can retrieve these from your downloaded data using one of the scripts in the amoebae/misc_scripts folder. First, let’s generate a query for the A. thaliana AP-1/2 beta subunit(s), which is a component of both the AP-1 and AP-2 complexes, using a representative sequence:

In [ ]:
%%bash
SECONDS=0

for QUERYFILE in temporary_query_dir/*.faa; do amoebae add_to_queries $QUERYFILE; done

ELAPSED="Preparing query sequences for searching took the following amount of time: $(($SECONDS / 3600))hrs $((($SECONDS / 60) % 60))min $(($SECONDS % 60))sec"
echo $ELAPSED

In [ ]:
%%bash
amoebae list_queries

## Construct alignments for profile similarity searching

In [ ]:
%%time

# Define a dictionary of NCBI sequence accessions and filenames to which to write the corresponding sequences.
query_title_dict = {"AP1beta": "NP_194077.1,CBI34366.3,XP_015631818.1,XP_024516549.1,OAE33273.1",
                    "AP2alpha": "NP_851058.1,XP_002270388.1,XP_015631820.1,PTQ35247.1,XP_024525508.1",
                    "AP2mu": "NP_974895.1,XP_002281297.1,XP_015627628.1,OAE25965.1,XP_002973295.1",
                    "AP2sigma": "NP_175219.1,XP_015618362.1,PTQ50284.1,XP_002275803.1,XP_024518676.1",
                    "Sec12": "NP_566961.1,XP_002262948.1,XP_015647566.1,OAE21792.1,XP_024530559.1",
                    "SNAP33": "NP_200929.1,XP_002284486.1,AAW82752.1,EFJ31467.1,OAE29824.1,XP_006270633.1,XP_006010378.1,XP_006625751.1,NP_001080510.1,XP_020370357.1,XP_015181699.1,XP_031769811.1",
                    "Rab2": "NP_193449.1,XP_003635585.2,XP_015626284.1,XP_002965710.1,PTQ28228.1"
                   }
                    

# Make a new temporary directory to store sequence files.
temp_alignment_dir_name = 'temporary_alignment_dir'
assert not os.path.isdir(temp_alignment_dir_name), """Directory already exists."""
os.mkdir(temp_alignment_dir_name)

# Download query sequences and write to multiple-sequence FASTA files.
for query_title in query_title_dict.keys():
    accession_list_string = query_title_dict[query_title]
    filepath = os.path.join(temp_alignment_dir_name, query_title + '_hmm1.faa')
    if not os.path.isfile(filepath):
        net_handle = Entrez.efetch(db="protein", id=accession_list_string, rettype="fasta", retmode="text")
        out_handle = open(filepath, "w")
        out_handle.write(net_handle.read())
        out_handle.close()
        net_handle.close()

In [ ]:
%%bash
SECONDS=0

for X in temporary_alignment_dir/*.faa; do amoebae align_fa $X --output_format fasta; done

ELAPSED="Aligning FASTA files took the following amount of time: $(($SECONDS / 3600))hrs $((($SECONDS / 60) % 60))min $(($SECONDS % 60))sec"
echo $ELAPSED

Similar to the AMOEBAE_Data/Genomes/0_genome_info.csv file, the AMOEBAE_Data/Queries/0_query_info.csv file contains information about each query file, which can be manually edited. One of the most important pieces of information is the "Query title". Different query files, such as a single FASTA sequence and an HMM, can have the same Query title if they are to be used to search for homologues or orthologues of the same protein(s).

# Print this notebook

In [ ]:
# Define author name for this notebook.
author_name = ""

# Define title of this notebook.
notebook_title = 'amoebae_tutorial_2'.replace('_', ' ')

In [ ]:
# Import modules.
import os
from string import Template

# Write a latex template file for converting this notebook to latex (as an intermediate to PDF).
latex_template_string = Template(r"""
((*- extends 'article.tplx' -*))

((* block author *))
\author{$an}
((* endblock author *))

((* block title *))
\title{$nt}
((* endblock title *))
""")
latex_file_contents =\
latex_template_string.substitute(an=author_name,
                                 nt=notebook_title
                                )
latex_template_file_path = 'latex_template.tplx'
with open(latex_template_file_path, 'w') as o:
    o.write(latex_file_contents)

# Convert notebook to PDF (with latex as an intermediate to process bibtex citations, etc.).
!jupyter nbconvert ../amoebae_tutorial_2.ipynb --to pdf --template {latex_template_file_path}

# Remove latex template file and bibtex file.
os.remove(latex_template_file_path)